# memstack-agent LLM 模块

本 notebook 演示 memstack-agent 的 LLM 抽象层。

## 概述

LLM 模块提供：
- **Message** - 不可变消息类型（system/user/assistant/tool）
- **ChatResponse** - 非流式响应
- **StreamChunk** - 流式响应块
- **ToolCall** - 工具调用
- **LLMConfig** - 不可变配置
- **LiteLLMAdapter** - 支持 100+ 提供商的适配器

## 1. 导入必要模块

In [ ]:
import sys
sys.path.insert(0, '/Users/tiejunsun/github/agi-demos/src')

from memstack_agent.llm import (
    # Types
    Message,
    MessageRole,
    ChatResponse,
    StreamChunk,
    ToolCall,
    Usage,
    # Config
    LLMConfig,
    anthropic_config,
    openai_config,
    gemini_config,
    deepseek_config,
    # Protocol
    LLMClient,
    # Adapter
    LiteLLMAdapter,
    create_llm_client,
)
import json

## 2. 消息类型 (Message)

Message 是不可变的聊天消息，支持 4 种角色。

In [ ]:
# 创建不同角色的消息
system_msg = Message.system("You are a helpful assistant.")
user_msg = Message.user("What is Python?")
assistant_msg = Message.assistant("Python is a programming language.")

print("System Message:")
print(json.dumps(system_msg.to_dict(), indent=2))
print("\nUser Message:")
print(json.dumps(user_msg.to_dict(), indent=2))

In [ ]:
# 带工具调用的助手消息
tool_call = ToolCall(
    id="call-001",
    name="search_web",
    arguments={"query": "Python programming"}
)

assistant_with_tool = Message.assistant(tool_calls=[tool_call])
print("Assistant with Tool Call:")
print(json.dumps(assistant_with_tool.to_dict(), indent=2))

In [ ]:
# 工具结果消息
tool_result = Message.tool_result(
    content='{"results": ["Python is...", "Python features..."]}',
    tool_call_id="call-001",
    name="search_web"
)

print("Tool Result Message:")
print(json.dumps(tool_result.to_dict(), indent=2))

## 3. 响应类型 (ChatResponse & StreamChunk)

In [ ]:
# 非流式响应
response = ChatResponse(
    content="Python is a high-level programming language.",
    tool_calls=[],
    finish_reason="stop",
    usage=Usage(prompt_tokens=20, completion_tokens=15, total_tokens=35),
    model="gpt-4"
)

print("ChatResponse:")
print(f"  Content: {response.content}")
print(f"  Has tool calls: {response.has_tool_calls}")
print(f"  Tokens: {response.usage.total_tokens}")

In [ ]:
# 流式响应块
chunks = [
    StreamChunk(delta="Python "),
    StreamChunk(delta="is "),
    StreamChunk(delta="awesome."),
    StreamChunk(delta="", finish_reason="stop"),
]

print("Stream Chunks:")
full_text = ""
for i, chunk in enumerate(chunks):
    full_text += chunk.delta
    is_final = "[FINAL]" if chunk.is_final else ""
    print(f"  {i+1}. delta='{chunk.delta}' {is_final}")

print(f"\nFull text: '{full_text}'")

## 4. 配置 (LLMConfig)

LLMConfig 是不可变配置，提供预设工厂函数。

In [ ]:
# 基本配置
config = LLMConfig(
    model="openai/gpt-4",
    api_key="sk-your-key",
    temperature=0.7,
    max_tokens=2048,
)

print("LLMConfig:")
print(f"  Model: {config.model}")
print(f"  Temperature: {config.temperature}")
print(f"  Max tokens: {config.max_tokens}")

In [ ]:
# 使用 with_* 方法创建新配置（不可变模式）
creative_config = config.with_temperature(0.9)
precise_config = config.with_temperature(0.1)

print("Original temperature:", config.temperature)
print("Creative temperature:", creative_config.temperature)
print("Precise temperature:", precise_config.temperature)

In [ ]:
# 预设配置
print("Preset Configurations:")
print(f"  Anthropic: {anthropic_config().model}")
print(f"  OpenAI: {openai_config().model}")
print(f"  Gemini: {gemini_config().model}")
print(f"  DeepSeek: {deepseek_config().model}")

## 5. LLM 协议 (LLMClient)

LLMClient 是一个 Protocol 接口，定义了所有 LLM 客户端必须实现的方法。

In [ ]:
# LLMClient 协议定义了两个核心方法：
# - generate(): 非流式生成
# - stream(): 流式生成

print("LLMClient Protocol Methods:")
print("  1. generate(messages, tools=None, **kwargs) -> ChatResponse")
print("  2. stream(messages, tools=None, **kwargs) -> AsyncGenerator[StreamChunk]")

## 6. LiteLLM 适配器

LiteLLMAdapter 实现了 LLMClient 协议，支持 100+ 提供商。

In [ ]:
# 使用工厂函数创建客户端
client = create_llm_client(
    model="openai/gpt-4",
    api_key="sk-your-key",  # 实际使用时替换为真实 API key
    temperature=0.7,
)

print(f"Created client for model: {client._config.model}")

In [ ]:
# 使用预设配置创建客户端
config = openai_config(model="gpt-4-turbo", api_key="sk-your-key")
client = LiteLLMAdapter(config)

print(f"Client model: {client._config.model}")

In [ ]:
# 验证适配器实现了协议
print(f"Is LLMClient: {isinstance(client, LLMClient)}")

## 7. 实际调用示例（需要 API Key）

以下代码展示了如何实际调用 LLM，需要设置有效的 API Key。

In [ ]:
import os
import asyncio

# 取消注释以下代码来实际调用 LLM
# os.environ["OPENAI_API_KEY"] = "your-api-key"

async def demo_generate():
    """演示非流式生成。"""
    client = create_llm_client(
        model="openai/gpt-4",
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    
    messages = [
        Message.system("You are a helpful assistant."),
        Message.user("What is 2 + 2?"),
    ]
    
    response = await client.generate(messages)
    print(f"Response: {response.content}")
    print(f"Tokens: {response.usage.total_tokens}")

# await demo_generate()  # 取消注释来运行

In [ ]:
async def demo_stream():
    """演示流式生成。"""
    client = create_llm_client(
        model="openai/gpt-4",
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    
    messages = [
        Message.user("Tell me a short joke."),
    ]
    
    print("Response: ", end="")
    async for chunk in client.stream(messages):
        print(chunk.delta, end="", flush=True)
    print()  # Newline

# await demo_stream()  # 取消注释来运行

## 8. 工具调用示例

In [ ]:
from memstack_agent.tools.protocol import ToolDefinition

# 定义工具
search_tool = ToolDefinition(
    name="search_web",
    description="Search the web for information",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query",
            },
        },
        "required": ["query"],
    },
)

async def demo_tool_calling():
    """演示工具调用。"""
    client = create_llm_client(
        model="openai/gpt-4",
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    
    messages = [
        Message.user("What's the weather in Tokyo?"),
    ]
    
    # 带工具的生成
    response = await client.generate(messages, tools=[search_tool])
    
    if response.has_tool_calls:
        for tc in response.tool_calls:
            print(f"Tool: {tc.name}")
            print(f"Arguments: {tc.arguments}")

# await demo_tool_calling()  # 取消注释来运行

## 9. 不可变性验证

In [ ]:
# 所有类型都是不可变的 (frozen dataclass)
msg = Message.user("Hello")

try:
    msg.content = "Changed"
except AttributeError as e:
    print(f"Message is immutable: {e}")

In [ ]:
config = LLMConfig(model="gpt-4")

try:
    config.model = "gpt-5"
except AttributeError as e:
    print(f"Config is immutable: {e}")

## 总结

本 notebook 演示了 memstack-agent 的 LLM 模块：

1. **Message** - 不可变消息类型（4 种角色）
2. **ChatResponse / StreamChunk** - 响应类型
3. **ToolCall / Usage** - 工具调用和用量追踪
4. **LLMConfig** - 不可变配置，支持 with_* 方法
5. **预设工厂** - anthropic_config, openai_config, gemini_config, deepseek_config
6. **LLMClient Protocol** - 统一接口
7. **LiteLLMAdapter** - 支持 100+ 提供商的实现
8. **create_llm_client** - 工厂函数

### 支持的提供商（通过 LiteLLM）

- OpenAI (GPT-4, GPT-3.5)
- Anthropic (Claude 3)
- Google (Gemini)
- DeepSeek
- Azure OpenAI
- AWS Bedrock
- Ollama (本地模型)
- 以及更多...